In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale as mms

In [11]:
def load_data(path,name):
    csv_path = os.path.join(path, name)
    return pd.read_csv(csv_path)

In [12]:
path = ""
name = "wine.data"
data = load_data(path,name)
print("Loading wine dataset ...")
data.head()

Loading wine dataset ...


,class,atr1,atr2,atr3,atr4,atr5,atr6,atr7,atr8,atr9,atr10,atr11,atr12,atr13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


# Pre-processing Data Set

In [13]:
print("Convert Categorical Output to One Hot Vector")
def categorical_output(): 
    Y = data["class"]
    Y = Y.values.reshape(-1,1)
    enc = preprocessing.OneHotEncoder(categories='auto')
    enc.fit(Y)
    Y = enc.transform(Y).toarray() #Converting in hot vectors
    return Y
Y = categorical_output()
print(Y.shape)

Convert Categorical Output to One Hot Vector
(178, 3)


In [14]:
print("Preparing inputs droping Labels and droping indexs")
def preprocessing_input():
    X = data
    X = X.drop("class", axis = 1)
    X = mms(X) #Scalling 
    return X
X = preprocessing_input()
print(X.shape)

Preparing inputs droping Labels and droping indexs
(178, 13)


# The MLP

In [15]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

In [16]:
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(1)
    
    W1 = np.random.normal(0, 1, (n_h, n_x)) #W1 -- weight matrix of shape (n_h, n_x)
    b1 = np.random.random((n_h, 1))         #b1 -- bias vector of shape (n_h, 1)
    W2 = np.random.normal(0, 1, (n_y, n_h)) #W2 -- weight matrix of shape (n_y, n_h)
    b2 = np.random.random((n_y, 1))         #b2 -- bias vector of shape (n_y, 1)


    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [23]:
def accuracy(Y_pred, Y):
    contador = 0
    Y_pred = Y_pred.T
    Y = Y.T
    for i in range(Y_pred.shape[0]) :
        if np.array_equal(Y_pred[i],Y[i]):
            contador +=1
    accuracy = (contador*100.0)/Y_pred.shape[0]
    return accuracy

In [17]:
def forward(X,Y,parameters,pred=False):
    
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    if pred:
        Y_prediction = np.zeros((Y.shape[0], Y.shape[1]))
        for i in range(A2.shape[0]):
            # Convert probabilities a[0,i] to actual predictions p[0,i]
            for j in range(A2.shape[1]):
                Y_prediction[i, j] = 1 if A2[i, j] >= 0.5 else 0
        return Y_prediction
    
    
    cache = {
             "Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2,cache

In [50]:
def compute_cost(A, Y):
    m = Y.shape[1] 
    cost = (- 1 / m) * np.sum(Y * np.log(A) + (1 - Y) * (np.log(1 - A)))
    return cost

In [38]:
def derivade_sigmoid(z):
    gz = sigmoid(z)
    return gz * (1-gz);

In [39]:
def backward(cache,parameters, X, Y):
    
    m = X.shape[1]
    
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    
    Z1 = cache["Z1"] 
    A1 = cache["A1"] 
    Z2 = cache["Z2"] 
    A2 = cache["A2"] 
    
    dZ2 = A2 - Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2,axis=1,keepdims=True)
    
    dZ1 = np.dot(W2.T, dZ2) * derivade_sigmoid(Z1)
    dW1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1,axis=1,keepdims=True)

    
    grads = {"dW1": dW1,
             "dW2": dW2,
             "db1": db1,             
             "db2": db2,
            }
    return grads

In [40]:
def update_parameters(parameters, grads, learning_rate=0.5):
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    
   
    dW1 = grads["dW1"]
    dW2 = grads["dW2"]
    db1 = grads["db1"]
    db2 = grads["db2"]


    W1 = W1 - learning_rate * dW1
    W2 = W2 - learning_rate * dW2
    b1 = b1 - learning_rate * db1
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [63]:
def nn_model(X_train, Y_train, X_test, Y_test, n_h, num_iterations=1):   
    n_x = X_train.shape[0]
    n_y = Y_train.shape[0] 

    # Initialize parameters, then retrieve W1, b1, W2, b2. Inputs: "n_x, n_h, n_y". Outputs = "W1, b1, W2, b2, parameters".
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    for i in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward(X_train,Y_train,parameters)
        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A2, Y_train)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward(cache, parameters, X_train, Y_train)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads)
        
        # Print the cost every 10 iterations
        #if i % 10 == 0:
         #   print ("Cost after iteration %i: %f" % (i, cost))
            
    # Predict test/train set examples
    Y_prediction_train = forward(X_train,Y_train,parameters,pred=True)
    Y_prediction_test  = forward(X_test ,Y_test,parameters,pred=True)

    # Print train/test Errors    
    print("Train Accuracy: ",accuracy(Y_prediction_train, Y_train)) 
    print("Test  Accuracy: ",accuracy(Y_prediction_test, Y_test))
    
    return parameters

# Divide data into Train and Test Sets

In [75]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.25,random_state=0)
n_h = 25
nn_model(X_train.T, Y_train.T, X_test.T, Y_test.T, n_h, num_iterations=1000)
print("Los datos estan en Shuffle")

Train Accuracy:  98.49624060150376
Test  Accuracy:  100.0
Los datos estan en Shuffle
